In [5]:
import cv2
import mediapipe as mp
import numpy as np

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b =  np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [ ]:
import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

# Function to listen for the "ready" command
def listen_for_ready():
    with sr.Microphone() as source:
        print("Say 'ready' to start the video feed.")
        audio = recognizer.listen(source)

    try:
        command = recognizer.recognize_google(audio)
        if "ready" in command.lower():
            return True
        else:
            return False
    except sr.UnknownValueError:
        print("Could not understand the audio")
        return False
    except sr.RequestError as e:
        print(f"Could not request results: {e}")
        return False

if listen_for_ready():
        print("Starting the video feed...")

In [7]:
def deep_squat_score(x, y, z):
    if x == True and y == True and z == True:
        return 3
    elif  x == True and y == True and z == False:
        return 2
    elif  x == False and y == False and z == False:
        return 1
    else:
        return 10

In [8]:
### DEEP SQUAT

import time
angles_dict = {}
knee_distance = {}
parallelism_angles = {}

min_l_hip_angle = 180
min_r_hip_angle = 180
squat_depth = False

starting_knee_distance = None
knee_distance = 1
knee_distance_threshold = .2 
knees_caved_in = False

min_torso_angle = 180
min_tibia_angle = 180

parallelism_threshold = 15
parallel = True

squat_score = 0


cap = cv2.VideoCapture(0)
start_time = time.time()
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
    
        # Detect stuff and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        # Make detection
        results = pose.process(image)
    
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                     
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            
             # Calculate angles
            torso_angle = calculate_angle(l_shoulder, l_hip, l_knee)
            tibia_angle = calculate_angle(l_hip, l_knee, l_ankle)

            # Find Minimum Angle
            if torso_angle < min_torso_angle:
                min_torso_angle = torso_angle
            
            parallelism_angles["torso_angle"] = torso_angle

            if tibia_angle < min_tibia_angle:
                min_tibia_angle = tibia_angle
            
            parallelism_angles["tibia_angle"] = tibia_angle
            
            angle_difference = abs(tibia_angle - torso_angle)
            
            # Checking if torso and tibia are parallel
            if angle_difference >= parallelism_threshold:
                parallel = False
            
            
            # Calculate Hip Angle
            l_hip_angle = calculate_angle(l_shoulder, l_hip, l_knee)
            r_hip_angle = calculate_angle(r_shoulder, r_hip, r_knee)
            
            #Loop and breakdownn into each frame and calculate hip angle
            if l_hip_angle < min_l_hip_angle:
                min_l_hip_angle = l_hip_angle
            
            angles_dict['min_l_hip_angle'] = min_l_hip_angle
            
            if r_hip_angle < min_r_hip_angle:
                min_r_hip_angle = r_hip_angle
            
            angles_dict['min_r_hip_angle'] = min_r_hip_angle
            
            # Checking if depth has been reached
            if (min_l_hip_angle + min_r_hip_angle)/2 <= 90:
                squat_depth = True
            
            # Calculate Distance Between Elbows
            knee_distance = np.linalg.norm(np.array(l_knee) - np.array(r_knee))

            # Record the starting elbow distance
            if starting_knee_distance is None:
                starting_knee_distance = knee_distance

            # Take Minimum Distance Between Knees
            if knee_distance < min_knee_distance:
                min_knee_distance = knee_distance

            knee_distance['min_knee_distance'] = min_knee_distance
            knee_distance['starting_knee_distance'] = starting_knee_distance
            
            # Check if knees have caved in 
            if (starting_knee_distance - min_knee_distance) > knee_distance_threshold:
                knees_caved_in = True
             
            # Visualize Angle          
            cv2.putText(image, str(r_hip_angle),
                       tuple(np.multiply(r_hip, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
                     
        except:
            pass 
    
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2, circle_radius = 2),
                                  mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2, circle_radius = 2))
       
        cv2.imshow("Mediapipe Feed", image)
        
        squat_score = deep_squat_score(squat_depth, parallel, knees_caved_in)
    
        # Calculate the elapsed time
        elapsed_time = time.time() - start_time
    
        # Check if 10 seconds have passed
        if elapsed_time >= 10:
            break
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
print(angles_dict)
print(knee_distance)
print(parallelism_angles)          
print(squat_score)

{'min_l_hip_angle': 68.17116817040343, 'min_r_hip_angle': 76.7715683796989}
0.21615920750475226
{'torso_angle': 68.17116817040343, 'tibia_angle': 62.89287079322714}
2


In [ ]:
def shoulder_mobility_score(x, y):
    if x <= (y * 1.0):
        return 3
    elif  x <= (y * 1.5) and x >= (y * 1.0):
        return 2
    elif  x >= (y * 1.5):
        return 1

In [ ]:
### SHOULDER MOBILITY EXERCISE
wrist_distance_dict = {}
min_wrist_distance = 100

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
    
        # Detect stuff and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    
        # Make detection
        results = pose.process(image)
    
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
           
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate Distance Between Landmarks
            wrist_distance = np.linalg.norm(np.array(l_wrist) - np.array(r_wrist))
            
            # Take Minimum Distance Between Wrists
            if wrist_distance < min_wrist_distance:
                min_wrist_distance = wrist_distance
            
            wrist_distance_dict['min_wrist_distance'] = min_wrist_distance
            
            


            # Draw the distance text on the frame
            cv2.putText(image, f"Wrist Distance: {distance_inches:.2f} units",
                tuple(np.multiply(l_wrist, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
       
        except:
            pass 
    
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2, circle_radius = 2),
                                  mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2, circle_radius = 2))
       
        shoulder_score = shoulder_mobility_score()
        
        cv2.imshow("Mediapipe Feed", image)
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()